In [1]:
import copy
import os
import time
import warnings

import aprofiles as apro
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

In [2]:
BASE_DIR_OUT = 'data/v-profiles'
date = '2021-09-09'
yyyy = date.split('-')[0]
mm = date.split('-')[1]
dd = date.split('-')[2]

In [3]:
datepath = os.path.join(BASE_DIR_OUT, yyyy, mm, dd, 'profiles')
onlyfiles = [f for f in os.listdir(datepath) if os.path.isfile(os.path.join(datepath, f))]

In [4]:
matching_index = [i for i, s in enumerate(onlyfiles) if "0-20000-0-01010" in s][0]

In [5]:
filename = onlyfiles[matching_index]
ds = xr.open_dataset(os.path.join(datepath, filename), decode_times=True)

In [8]:
# calculate the max extinction and determine the scene for each hour of the day
# need to convert time (from int to datetime) in order to use the resample method from xarray
ds = ds.assign_coords(time = ds.time.data.astype("datetime64[ms]"))

# in order to prevent some monotony issue, sort by time
ds = ds.sortby('time')

# extinction for each hour
max_ext_profiles = ds.extinction.resample(time="1H").max().data

# take the maximum value in each profile
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    max_ext = np.nanmax(max_ext_profiles, axis=1)

# scene for each hour
# attribute a weight to each scene in order to prioritize the scenes
scene_weights = {'foc': 4, 'low_cloud': 3, 'mid_cloud': 2, 'high_cloud': 1, 'aer': 0}
scene = np.asarray(ds.scene.data)
for scene_weight in scene_weights.keys():
    scene[scene==scene_weight] = scene_weights[scene_weight]

# creates dataarrays in order to resample with xarray method
ds["weight_scene"] = ('time', scene)
# takes the highest weight for resampled data
max_weight = ds.scene.resample(time="1H").max().data
# some weight might be nan if there was no measurements available
max_weight = [weight if not np.isnan(weight) else None for weight in max_weight]

# reverse dictionnary
weight_scenes = {v: k for k, v in scene_weights.items()}
max_scene =[weight_scenes[weight] if weight is not None else None for weight in max_weight]

In [9]:
max_scene

['low_cloud',
 'low_cloud',
 'low_cloud',
 'low_cloud',
 'low_cloud',
 'low_cloud',
 'low_cloud',
 'low_cloud',
 'low_cloud',
 'aer',
 'low_cloud',
 'low_cloud',
 None,
 None,
 'low_cloud',
 'aer',
 'low_cloud',
 'low_cloud',
 'low_cloud',
 'low_cloud',
 'low_cloud',
 'low_cloud',
 'low_cloud',
 'low_cloud']

In [6]:
# calculate the max extinction and determine the scene for each hour of the day
# need to convert time (from int to datetime) in order to use the resample method from xarray
ds = ds.assign_coords(time = ds.time.data.astype("datetime64[ms]"))

# in order to prevent some monotony issue, sort by time
ds = ds.sortby('time')

# extinction for each hour
max_ext_profiles = ds.extinction.resample(time="1H").max().data

# take the maximum value in each profile
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    max_ext = np.nanmax(max_ext_profiles, axis=1)

# scene for each hour
# attribute a weight to each scene in order to prioritize the scenes
scene_weights = {'foc': 4, 'low_cloud': 3, 'mid_cloud': 2, 'high_cloud': 1, 'aer': 0}
scene = np.asarray(ds.scene.data)

In [7]:
for s in scene:
    print(s)

aer
aer
low_cloud
aer
aer
aer
aer
aer
aer
aer
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
aer
low_cloud
low_cloud
aer
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
aer
aer
low_cloud
aer
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
aer
aer
aer
aer
aer
aer
aer
aer
low_cloud
aer
aer
aer
aer
aer
aer
aer
aer
aer
low_cloud
aer
aer
aer
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
aer
aer
aer
aer
aer
aer
aer
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
aer
low_cloud
low_cloud
low_cloud
low_cloud
aer
low_cloud
aer
aer
low_cloud
aer
aer
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
low_cloud
aer
low_cloud
aer
aer
low_cloud
low_cloud
low_cloud
low_cloud
aer
low_cloud
low_cloud
low_cloud
low_cloud
aer
aer
low_cloud
low_cloud
aer
aer
aer
aer
aer
aer
aer
low_cloud
aer
aer
low_cloud
aer
aer
aer


In [1]:
vars = 'extinction_scene'

In [2]:
vars.split('_')

['extinction', 'scene']